In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt
import pickle 
import idaes

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
strategy3 = True

In [3]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [4]:
Nt =110

if strategy3:
    static_ind = [0,1,2,3,4,5,6,7,8,9,10]
    dynamic_ind = [3,7,11,12,13]

else:
    static_ind = [0,1,2,4,5,6,8,9,10]
    dynamic_ind = [3,7,11,12,13]

In [5]:
dataObject = DataProcess()
dataObject.read_jaco('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.stack_Q(static_ind, dynamic_ind, Nt)


jacobian shape: (1540, 5)


In [6]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]

if strategy3:
    static_idx_upd = [0,1,2,3,4,5,6,7,8,9,10]
    dynamic_idx_upd = [9,10,11,12,13]
else:
    static_idx_upd = [0,1,2,3,4,5,6,7,8]
    dynamic_idx_upd = [9,10,11,12,13]

num_param = 5

measure_names = static_name + dynamic_name
print(measure_names)

['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']


In [7]:
calculator = MeasurementOptimizer(Q, static_idx_upd, dynamic_idx_upd, 
                                  num_param, measure_names=measure_names, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


In [ ]:
print(calculator.fim_collection[0])

In [ ]:
def check(fim_set):
    for i in range(num_total):
        for j in range(num_total):
            fim = fim_set[i*num_total+j]
            if np.any(fim):
                logdet = np.log(np.linalg.det(fim))
                if logdet < 0:
                    print("unit FIM with small Det: ", i, j, ', logdet:', logdet)
                if max(np.linalg.eigvals(fim))/min(np.linalg.eigvals(fim)) > 1000:
                    print("unit FIM with big condition number:", i, j, ", eigvals:", np.linalg.eigvals(fim))
num_total = 121

In [ ]:
def print_fim(t):
    fim = calculator.fim_collection[t*num_total+t]
    print(fim)
    print(np.linalg.eigvals(fim))

total_fim = np.zeros((5,5))
for i in range(314721):
    total_fim += calculator.fim_collection[i]
print(total_fim)

In [ ]:
#check(calculator.fim_collection)

### Reform the inputs

In [8]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [9]:
if strategy3:
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        7000,
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
        7000,
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]
else:
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

for i in range(Nt*num_dynamic):
    cost.append(100)

install_cost_dynamic = [100, 100, 500, 500, 500]

In [10]:
print(sum(cost[:11])+sum(install_cost_dynamic)+20*200)

27700


In [12]:
with open('Mar20_A_110_20000_strategy3_mip', 'rb') as f:
    x = pickle.load(f)
    
print(x)

init_cov_y = x.copy()

for i in range(11):
    init_cov_y[i,i] = 1

[[1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
# initialize all decisions
#init_cov_y = np.ones(x.shape) 

In [33]:
#with open('strategy3_all_covy', 'rb') as f:
with open('FIM_strategy3_A_110_20000', 'rb') as f:
    fim_prior = pickle.load(f)
    
print(fim_prior)

[[ 5.54781941e+03 -2.79896951e+00 -4.14297808e+03  1.09902329e+04
  -4.94072165e+02]
 [-2.79896951e+00  5.20826025e+00  7.60848042e+00  5.06824563e+01
  -1.04261168e+01]
 [-4.14297808e+03  7.60848042e+00  3.32483923e+03 -8.52165366e+03
   1.68122219e+02]
 [ 1.09902329e+04  5.06824563e+01 -8.52165366e+03  2.52864273e+04
  -8.96811655e+02]
 [-4.94072165e+02 -1.04261168e+01  1.68122219e+02 -8.96811655e+02
   3.67315096e+02]]


In [ ]:
sol = x.diagonal()

num_time = 110
num_measure = 16
num_static = 11
sol_2d = np.zeros((num_measure,num_time))

for i in range(num_measure):
    for j in range(num_time):
        if i<num_static:
            sol_2d[i,j] = sol[i]
        else:
            #print((i-num_static)*num_time+j)
            sol_2d[i,j] = sol[(i-num_static)*num_time+j+num_static]
            
print(sol_2d)

# Reorganize sol_2d, so that static and dynamic z are together 
sol_2d_reorg = []
static_idx = [3,7]

for i in range(11):
    if i not in static_idx:
        sol_2d_reorg.append(sol_2d[i])
    
sol_2d_reorg.append(sol_2d[3])
sol_2d_reorg.append(sol_2d[7])

for j in range(11,16):
    sol_2d_reorg.append(sol_2d[j])
    
sol_2d_reorg = np.asarray(sol_2d_reorg)
#sol_2d = sol_2d[:3,:]+sol_2d[4:7,:]+sol_2d[8:11,:]+sol_2d[3,:]+sol_2d[7,:]+sol_2d[11:,:]

print(sol[:11])
print(sol[11:121])
print(sol_2d_reorg[11])

In [34]:
if strategy3:
    num_fixed_opt = 11
else:
    num_fixed_opt = 9

num_dynamic_opt = 5

mip_option = False
objective = "D"
sparse_opt = True
fix_opt = False

each_manual_num = 5
#each_manual_num = None
#total_manual_num = None
total_manual_num = 20
budget_opt = 27700

discretize_time = 10
num_dynamic_time = np.linspace(2,220,Nt)
    
#print(num_dynamic_time)

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 
    
#print(dynamic_time_dict)

In [35]:
mod = calculator.continuous_optimization(cost, mixed_integer=mip_option, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    discretize_time = discretize_time, 
                    manual_number = total_manual_num, budget=budget_opt,
                    dynamic_install_cost = install_cost_dynamic, 
                    each_manual_number = each_manual_num,  
                    init_cov_y= init_cov_y, initial_fim = fim_prior)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

{'ele_0_0': 5547.819413862037, 'ele_0_1': -2.7989695079930907, 'ele_0_2': -4142.978078958834, 'ele_0_3': 10990.232885489535, 'ele_0_4': -494.07216522961767, 'ele_1_1': 5.208260254418963, 'ele_1_2': 7.608480416183696, 'ele_1_3': 50.68245631431674, 'ele_1_4': -10.426116776757201, 'ele_2_2': 3324.8392276658083, 'ele_2_3': -8521.653655909617, 'ele_2_4': 168.12221904300958, 'ele_3_3': 25286.42731937437, 'ele_3_4': -896.8116552992137, 'ele_4_4': 367.3150958511481}
initialize with: {'ele_0_0': 5547.819413862037, 'ele_0_1': -2.7989695079930907, 'ele_0_2': -4142.978078958834, 'ele_0_3': 10990.232885489535, 'ele_0_4': -494.07216522961767, 'ele_1_1': 5.208260254418963, 'ele_1_2': 7.608480416183696, 'ele_1_3': 50.68245631431674, 'ele_1_4': -10.426116776757201, 'ele_2_2': 3324.8392276658083, 'ele_2_3': -8521.653655909617, 'ele_2_4': 168.12221904300958, 'ele_3_3': 25286.42731937437, 'ele_3_4': -896.8116552992137, 'ele_4_4': 367.3150958511481}
initialized
initialized
initialized
initialized
initializ


 Consider M =
 [[ 5.58526853e+03 -2.97326805e+00 -4.17294391e+03  1.10631409e+04
  -4.95540720e+02]
 [-2.97326805e+00  5.22992778e+00  7.64351677e+00  5.06377137e+01
  -1.03599022e+01]
 [-4.17294391e+03  7.64351677e+00  3.34827778e+03 -8.58228687e+03
   1.69345016e+02]
 [ 1.10631409e+04  5.06377137e+01 -8.58228687e+03  2.54427274e+04
  -8.97575175e+02]
 [-4.95540720e+02 -1.03599022e+01  1.69345016e+02 -8.97575175e+02
   3.66911982e+02]]
[5585.26852864115, -2.973268053241842, -4172.943912361064, 11063.140919225085, -495.5407199664204, 5.229927780127614, 7.643516766201278, 50.63771370470927, -10.359902161217407, 3348.277781210094, -8582.286866519556, 169.34501637304191, 25442.727408888386, -897.5751749746203, 366.91198247363906]
   logdet =  28.340009226870794 

Eigvals: [3.34378209e+04 8.27065835e+02 4.34612717e+02 4.51738604e+01
 3.74232879e+00]

 Consider M =
 [[ 5.56201880e+03 -3.04351432e+00 -4.15577277e+03  1.10154157e+04
  -4.90452835e+02]
 [-3.04351432e+00  5.20349292e+00  7.679


 Consider M =
 [[ 5.52603471e+03 -3.48543892e+00 -4.13345748e+03  1.09396404e+04
  -4.76161772e+02]
 [-3.48543892e+00  5.14498500e+00  7.74373603e+00  4.85267424e+01
  -9.24109418e+00]
 [-4.13345748e+03  7.74373603e+00  3.31789909e+03 -8.50102268e+03
   1.65537831e+02]
 [ 1.09396404e+04  4.85267424e+01 -8.50102268e+03  2.51470583e+04
  -8.51481260e+02]
 [-4.76161772e+02 -9.24109418e+00  1.65537831e+02 -8.51481260e+02
   3.39300395e+02]]
[5526.034712006184, -3.485438924275217, -4133.457483365157, 10939.640421991564, -476.16177216369886, 5.144984995130925, 7.743736029938226, 48.52674237358, -9.241094178233123, 3317.8990948018013, -8501.022682572913, 165.53783103519677, 25147.058334177622, -851.4812604735026, 339.30039485900693]
   logdet =  28.217305596900598 

Eigvals: [3.30638992e+04 8.16255489e+02 4.07501160e+02 4.40737077e+01
 3.70792974e+00]

 Consider M =
 [[ 5.52619417e+03 -3.48563446e+00 -4.13357907e+03  1.09399584e+04
  -4.76172718e+02]
 [-3.48563446e+00  5.14507339e+00  7.7439


 Consider M =
 [[ 5.54225429e+03 -3.13579052e+00 -4.14212430e+03  1.09725613e+04
  -4.86317376e+02]
 [-3.13579052e+00  5.17873460e+00  7.72725777e+00  4.93668751e+01
  -9.87705174e+00]
 [-4.14212430e+03  7.72725777e+00  3.32451237e+03 -8.51884420e+03
   1.66716386e+02]
 [ 1.09725613e+04  4.93668751e+01 -8.51884420e+03  2.52201504e+04
  -8.72790667e+02]
 [-4.86317376e+02 -9.87705174e+00  1.66716386e+02 -8.72790667e+02
   3.56135061e+02]]
[5542.254285657392, -3.1357905175615066, -4142.124295006683, 10972.56132520215, -486.31737648906204, 5.178734601774928, 7.727257770224433, 49.36687512112578, -9.877051739700077, 3324.512368824404, -8518.844195636299, 166.7163855887543, 25220.150418845842, -872.7906669571113, 356.1350607430939]
   logdet =  28.284632921097142 

Eigvals: [3.31573207e+04 8.18127831e+02 4.24108213e+02 4.49572533e+01
 3.71691776e+00]

 Consider M =
 [[ 5.54268869e+03 -3.10695759e+00 -4.14215700e+03  1.09735129e+04
  -4.87071165e+02]
 [-3.10695759e+00  5.18053185e+00  7.7261


 Consider M =
 [[ 5.54587775e+03 -2.92913539e+00 -4.14229250e+03  1.09799012e+04
  -4.91981350e+02]
 [-2.92913539e+00  5.19001168e+00  7.72346635e+00  4.97589387e+01
  -1.01958508e+01]
 [-4.14229250e+03  7.72346635e+00  3.32453684e+03 -8.51912833e+03
   1.66954605e+02]
 [ 1.09799012e+04  4.97589387e+01 -8.51912833e+03  2.52343153e+04
  -8.84242548e+02]
 [-4.91981350e+02 -1.01958508e+01  1.66954605e+02 -8.84242548e+02
   3.65400044e+02]]
[5545.877747053706, -2.929135387024371, -4142.292504379183, 10979.901214533757, -491.98134987697415, 5.190011682244748, 7.723466347129526, 49.75893874824232, -10.195850792787791, 3324.536837701323, -8519.128334123445, 166.95460469691884, 25234.31525051627, -884.2425479787581, 365.400044444849]
   logdet =  28.314692242216807 

Eigvals: [3.31745021e+04 8.18226035e+02 4.33592643e+02 4.52817619e+01
 3.71733472e+00]

 Consider M =
 [[ 5.54658320e+03 -2.89292248e+00 -4.14234591e+03  1.09813131e+04
  -4.92976496e+02]
 [-2.89292248e+00  5.19182778e+00  7.7211


 Consider M =
 [[ 5.54890666e+03 -2.76555818e+00 -4.14215981e+03  1.09849006e+04
  -4.95771872e+02]
 [-2.76555818e+00  5.19924046e+00  7.73833723e+00  5.00018913e+01
  -1.03723496e+01]
 [-4.14215981e+03  7.73833723e+00  3.32467029e+03 -8.51921596e+03
   1.67017985e+02]
 [ 1.09849006e+04  5.00018913e+01 -8.51921596e+03  2.52426361e+04
  -8.91092242e+02]
 [-4.95771872e+02 -1.03723496e+01  1.67017985e+02 -8.91092242e+02
   3.71128139e+02]]
[5548.906662381779, -2.765558183818952, -4142.159811400112, 10984.900596206255, -495.7718724813769, 5.199240457264445, 7.7383372281010745, 50.00189130271123, -10.37234963778509, 3324.6702855702006, -8519.215964569266, 167.01798464279014, 25242.6360774701, -891.0922420016956, 371.12813867079694]
   logdet =  28.33942003796757 

Eigvals: [3.31851362e+04 8.18169341e+02 4.39791103e+02 4.57243492e+01
 3.71940067e+00]

 Consider M =
 [[ 5.54913008e+03 -2.75263766e+00 -4.14212681e+03  1.09851938e+04
  -4.96008558e+02]
 [-2.75263766e+00  5.20004841e+00  7.7413


 Consider M =
 [[ 5.55033746e+03 -2.69922499e+00 -4.14196272e+03  1.09866211e+04
  -4.97460184e+02]
 [-2.69922499e+00  5.20637503e+00  7.72929758e+00  5.01425324e+01
  -1.04361531e+01]
 [-4.14196272e+03  7.72929758e+00  3.32490495e+03 -8.51959819e+03
   1.66958785e+02]
 [ 1.09866211e+04  5.01425324e+01 -8.51959819e+03  2.52457772e+04
  -8.93561030e+02]
 [-4.97460184e+02 -1.04361531e+01  1.66958785e+02 -8.93561030e+02
   3.73702239e+02]]
[5550.3374608144295, -2.6992249877948065, -4141.962715978089, 10986.62110830417, -497.460184089889, 5.206375027289955, 7.729297577845058, 50.14253238708305, -10.436153057893005, 3324.9049464109908, -8519.598186333782, 166.9587852958502, 25245.777150053807, -893.5610303560239, 373.7022393021858]
   logdet =  28.35430255820119 

Eigvals: [3.31892465e+04 8.18189788e+02 4.42716750e+02 4.60521118e+01
 3.72297557e+00]

 Consider M =
 [[ 5.55038187e+03 -2.69757460e+00 -4.14195157e+03  1.09866579e+04
  -4.97510694e+02]
 [-2.69757460e+00  5.20704818e+00  7.7259

In [36]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.55068912e+03 -2.68836024e+00 -4.14186590e+03  1.09868809e+04
  -4.97871713e+02]
 [-2.68836024e+00  5.20999602e+00  7.71292427e+00  5.02117209e+01
  -1.04494187e+01]
 [-4.14186590e+03  7.71292427e+00  3.32507884e+03 -8.51999675e+03
   1.66913944e+02]
 [ 1.09868809e+04  5.02117209e+01 -8.51999675e+03  2.52470182e+04
  -8.94049930e+02]
 [-4.97871713e+02 -1.04494187e+01  1.66913944e+02 -8.94049930e+02
   3.74307784e+02]]
trace: 34502.30395146165
det: 2072035598343.0054
[3.31906521e+04 8.18273778e+02 4.43473122e+02 4.61795786e+01
 3.72533664e+00]
Pyomo OF: 28.359553508340188
Log_det: 28.359552620845673


In [37]:
ans_y, sol_y = calculator.extract_solutions(mod)

Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  1.0
Ads.gas_outlet.T :  1.0
Ads.gas_outlet.z("CO2") :  1.0
Des.gas_inlet.F :  1.0
Des.gas_outlet.F :  1.0
Des.gas_outlet.T :  1.0
Des.gas_outlet.z("CO2") :  1.0
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  1.0
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2")
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.15 0.16 0.19 0.28 0.23 0.15
 0.16 0.19 0.28 0.23 0.15 0.16 0.19 0.28 0.23 0.15 0.16 0.09 0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
Des.gas_outlet.z("CO2")
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.

In [ ]:
print(ans_y)

In [30]:
# verify cost 
real_cost = 0 
for i in range(num_total):
    real_cost += ans_y[i,i]*cost[i]
    
dynamic_inst = []
for j in range(num_static, num_dynamic+num_static):
    dynamic_inst.append(pyo.value(mod.if_install_dynamic[j]))
#    for t in range(Nt):
#        if sol_y[j,t]>0.99:
#            dynamic_inst[j] = 1 
            
real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
print(dynamic_inst)
print('Real cost:', real_cost)
print('pyomo calculated cost:', pyo.value(mod.cost))

[0.9999999019597795, 0.9999999019522361, 0.9999999019854565, 0.9999999019836571, 0.9999999019833512]
Real cost: 24925.999833367434
pyomo calculated cost: 25699.942467044617


In [ ]:
count = 0 
for i in range(561):
    for j in range(i,561):
        #if pyo.value(mod.cov_y[i,j])<0.96:
        print(pyo.value(mod.cov_y[i,j]))
            #count += 1 
            
print(count)

In [31]:
store = True

if store:
    file = open('Mar30_D_110_27700_strategy3', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('FIM_Mar30_D_110_27700', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()